This notebook shows an example of an evolutionary algorithm in practice. First load necessary packages.

In [ ]:
import random as rd
import numpy as np
import time

The N-queens problem is the problem of placing N queens on a chessboard of NxN such that no queen can go and capture another queen. For the non-chess players: A queen may move vertically, horizontally and diagonally, as many moves as desired. Below is an example of a correct solution for N=8.

> ![alt text](https://cdn-images-1.medium.com/max/1200/1*Zm2pbDR5CS2w2xeUbTBxQQ.png)

For N=8, there is a total of 64 choose 8 = 4,426,165,368 possible ways to place the queens, of which only 92 are a correct solution. So this already makes it quite impractical to check all the ways, let alone when N is larger. Therefore, we can use an evolutionary algorithm to find a solution.





An evolutionary algorithm consists of a number of steps that are run iteratively until a solution is reached that meets a certain criteria. This can be a fixed number of iterations, or until no improvement is seen for x number of iterations, or until a solution is reached (as in the N-queens problem).


> ![alt text](https://cdn-images-1.medium.com/max/1600/1*odW0CYMTeS-R5WW1hM0NUw.jpeg)

Before starting, it is good to think about how we want to represent a solution. In the case of the N-queens problem, we can choose an 8x8 matrix, with a 1 if there is a queen, and a 0 if there is none. For example


```
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
```


as in the chessboard from above. However, this gives a total of 2^64 possibilities, which is quite a lot to go through. We can also be a little smarter: We take an 8x1 vector, with a number between 1 and 8 in each entry, indicating the row number in which the queen is located. This way we automatically ensure that there are no queens on the same columns. Now there are only 8^8 possibilities left. Of course, we can even go a bit further: now we take an 8x1 vector, with each entry a number between 1 and 8, where each number may only occur once: Similarly, there cannot be two queens in each row and we are left with 8! = 40320 options left, significantly less! The above example is then represented by

```
[1, 7, 4, 6, 8, 2, 5, 3]
```

We call such a representation an 'individual'. For other problems you can also use other representations, this depends on the problem. For an evolutionary algorithm to work, you need a whole population, consisting of a lot of individuals. The size of the population is a parameter, for different problems a different population size works. In addition, of course, we need to know how "good" an individual is. For that we use a fitness function, in this particular case we count the number of queens that can beat each other.

---

Let's start by creating a class for an individual. We add two methods:


*   One to randomly create an individual
*   One to calculate the fitness of an individual





In [ ]:
# Initiate class individual
class Individual:
    def __init__(self, length, genes = None):
        self.length = length
        self.fitness = None
        if genes is not None:
            self.genes = genes
        else:
            self.genes = []*length


    # Randomly initialise the genes
    def InitialiseRandom(self):
        self.genes = rd.sample(range(self.length), self.length)

    # Create a fitness function to calculate how "good" the current individual is
    def GetFitness(self, N):
        """
        Calculates an individual's fitness
        :param self: individual
        """
        fit = 0
        for i in range(N):
            """
            Count the number of queens that can be beaten by this queen.
            Because of the encoding, only the diagonal needs to be looked at.
            A queen can be beaten by another queen if the absolute difference in
              the genes is equal to the absolute difference in positions.
            """
            for j in range(N):
                # A queen cannot beat itself
                if i != j:
                    if abs(self.genes[i] - self.genes[j]) == abs(i - j):
                        fit = fit + 1
        self.fitness = fit

So to begin with, we create a population of randomly created individuals. In addition, we set some parameters for the size of the chessboard and for the size of the population.

In [ ]:
# Global parameters
population_size = 1000
N = 14 # The chessboard's size

# Initialize a population
population = [Individual(N) for i in range(population_size)]
for i in population:
    # Randomly initialize each individual's genes and calculate their fitness.
    i.InitialiseRandom()
    i.GetFitness(N)

Next, we must determine which individuals are good. Indeed, better individuals should be given a higher chance to "propagate", just like in nature. For this we use the fitness function.

Then there are different ways of selecting parents that are used to create a new population. Below we use tournament selection: A number of individuals are randomly selected, and the "strongest" individual from these is added to the parent population. This is repeated until the number of desired parents is reached.

Other ways to do parent selection are for example:

* Fitness Proportionate Selection
* Rank Selection
* Random Selection
* Reward Based Selection
* Age Based Selection
* Elitism

It depends on the problem which ways work well and which ways work less well.

In [ ]:
# A parent selection mechanism example
def TournamentSelection(pop, size, tournament_size):
    """
    Creates a population of parents based on tournament selection: tournament_size
    individuals are selected from pop and the one with the highest fitness wins
    and is added to the parent population.
    :param pop: The population used to select parents from
    :param size: the size of the parent population
    :param tournament_size: The size of a tournament. Must be smaller than population_size
    :return: the parent population
    """
    parent_pop = []
    for i in range(size):
        # select tournament_size individuals (random sampling without replacement)
        tournament = rd.sample(pop, tournament_size)
        # select the best individual
        winner = np.argmin([i.fitness for i in tournament])
        # add it to parent_pop
        parent_pop.append(pop[winner])
    return(parent_pop)

Next, it is time to create new children. Below we use crossover: One part of one parent is used, and another part of the other (see below).

![alt text](https://cdn-images-1.medium.com/max/1600/1*YhmzBBCyAG3rtEBbI0gz4w.jpeg)

Here as well, there are other options:

*   Single point crossover
*   k-point crossover
*   Uniform crossover
*   Edge recombination



In [ ]:
def CrossOver(parent1, parent2, k, l):
    """
    crossover applied to two parents
    :param parent1: parent 1
    :param parent2: parent 2
    :param k l: breakpoints of the genen. Before k will be used of parent1, afterwards of parent2
    :return: a child with combined genes
    """

    child = []
    # Select the to-be-used genes of both parents
    childcenter = [parent1.genes[j] for j in range(k,l+1)]
    childedges = [parent2.genes[j] for j in range(0,N) if parent2.genes[j] not in childcenter]
    # Combine the genes
    child[:k] = childedges[:k]
    child[k:l+1] = childcenter
    child[l+1:] = childedges[k:]

    return(child)

To discover new regions, we can add mutation on the children. Below we apply a swap operation.

![alt text](http://www.wardsystems.com/manuals/genehunter/_bm53.png)

Two randomly selected genes are swapped. Again, there are other possibilities, such as:

*   Bit flip mutation
*   Uniform mutation
*   Gaussian mutation
*   Bit string mutation
*   Random Resetting
*   Insert
*   Scramble
*   Inversion
*   2-opt
*   k-opt

Here it also depends on the representation which you can use. In addition, it also depends on the nature of the problem which mutations work well and which ones work badly.



In [ ]:
def Swap(child, factor):
    """
    Swap two genes
    :param child: child to be mutated
    :param factor: in (1,2), will we swap one or two genes?
    :return: mutated child
    """

    for i in range(factor):
        (k, l) = rd.sample(range(N), 2)

        # swap two genes
        old = child[k]
        child[k] = child[l]
        child[l] = old

    return(child)

Thus, a lot of children can be created. Of course, some children will have lower fitness than their parents, but some will also have higher fitness. By applying the survival of the fittest principle, better children have a higher chance of reproducing, so in the long run the whole population will become better and stronger genes will survive.

Herein it is important to make a good trade-off between "exploration" and "optimization". A high mutation factor and a lot of crossover will ensure that many new areas are discovered, but if the selection pressure is too low (less fit individuals get too high a chance to reproduce), the population will not improve. On the other hand, if the selection pressure is too high, the algorithm may get into a local optimum too quickly, so other good solutions may be missed.

Below is a description of the entire evolutionary cycle. The algorithm stops when the fitness of the best individual is 0, that is, when a solution has been reached.

In [ ]:
def GetBest(population):
    """
    :param: the current population
    :return: return the best individual of the current population
    """
    best = np.argmin([i.fitness for i in population])
    return(population[best])

# Global parameters
tournament_size = 20 # Tournament size
parent_size = 500 # Parent set size
mutation_factor = 0.8 # The chance of a single mutation
mutation_factor_double = 0.5 # The chance of a double mution, in case of a single

# Evolutionary cycle
i = 0
start_time = time.time()
while GetBest(population).fitness > 0:
    if i%10 == 0:
        print("iteratie", i)
    # Select parents
    parents = TournamentSelection(population, parent_size, tournament_size)
    childs = []
    # Add the best individual of the previous population (Elitism)
    childs.append(GetBest(population))
    while len(childs) < population_size:
        # Select parents from the parent set
        parent1 = rd.sample(parents, 1)[0]
        parent2 = rd.sample(parents, 1)[0]
        # Create two children (opposites of each other)
        (a,b) = rd.sample(range(N),2)
        k = min(a,b)
        l = max(a,b)
        child1 = CrossOver(parent1, parent2, k, l)
        child2 = CrossOver(parent2, parent1, k ,l)

        if bool(np.random.binomial(1, mutation_factor)):
            if bool(np.random.binomial(1, mutation_factor_double)):
                # Double mutation
                child1 = Swap(child1, 2)
            else:
                # Single mutation
                child1 = Swap(child1, 1)
        if bool(np.random.binomial(1, mutation_factor)):
            if bool(np.random.binomial(1, mutation_factor_double)):
                # Double mutatino
                child2 = Swap(child2, 2)
            else:
                # Single mutation
                child2 = Swap(child2, 1)
        # Append children
        childs.append(Individual(N, genes= child1))
        childs.append(Individual(N, genes= child2))

    # Update the population
    population = childs
    # Calculate the new population's fitness
    [i.GetFitness(N) for i in population]
    i = i + 1

end_time = time.time() - start_time
print("Solution found")

Finally: print the solution

In [ ]:
print("Solution found: ", GetBest(population).genes)
print("After " + str(i) + " iterations and " + str(end_time) + " seconds.")

So the code above works pretty fast considering there are N! configurations. By playing with the parameter settings, it could probably be even faster.

Above, the basics of an evolutionary algorithm are described. Of course, many extensions and modifications are possible. One can think of

* Parameters can change over time
* Parameters can also evaluate
* Why limit yourself to the laws of nature? 3 or more parents on a computer is also possible
* Different crossover and mutation methods in the same algorithm
* etc.